In [1]:
import glob
import codecs
import numpy
from pandas import DataFrame
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, f1_score

# สร้าง data ด้วย pandas DataFrame

In [2]:
d = [{'text':'London bridge is falling down','class':'london'}]
d.append({'text':'japan samurai universal studio spider man','class':'japan'});
d.append({'text':'china beijing','class':'china'});
d.append({'text':'thai chiangmai','class':'thailand'});
d.append({'text':'universal studio hollywood','class':'usa'});

# convert เป็น DataFrame   ดึงมาเฉพาะ field text

In [3]:
data = DataFrame(d)

In [4]:
arr_text=data.iloc[:]['text'].values

# CountVectorizer ทำ tokenize

In [6]:
vect = CountVectorizer(stop_words='english', lowercase=True)

# เริ่มตัดคำ

In [10]:
count_train = vect.fit(arr_text)

# ดู vocabulary

In [11]:
vect.vocabulary_

{'london': 7,
 'bridge': 1,
 'falling': 4,
 'japan': 6,
 'samurai': 9,
 'universal': 13,
 'studio': 11,
 'spider': 10,
 'man': 8,
 'china': 3,
 'beijing': 0,
 'thai': 12,
 'chiangmai': 2,
 'hollywood': 5}

# ทำ transform ให้อยู่ในรูปแบบ One Hot Encoding

In [12]:
transformer = vect.transform(arr_text)

# ดูผลลัพท์ matrix ที่ได้โดย array จะประกอบด้วย context ใน 1 paragraph หากมี word ใน context ให้ค่าเป็น 1 และหากไม่มี word ใน context ให้ค่าเป็น 0

In [13]:
print(transformer.toarray())

[[0 1 0 0 1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 1 1 1 1 0 1]
 [1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1]]


In [14]:
print(transformer)

  (0, 1)	1
  (0, 4)	1
  (0, 7)	1
  (1, 6)	1
  (1, 8)	1
  (1, 9)	1
  (1, 10)	1
  (1, 11)	1
  (1, 13)	1
  (2, 0)	1
  (2, 3)	1
  (3, 2)	1
  (3, 12)	1
  (4, 5)	1
  (4, 11)	1
  (4, 13)	1


In [15]:
print ('Shape of Sparse Matrix: ', transformer.shape)

Shape of Sparse Matrix:  (5, 14)


In [16]:
print ('Amount of Non-Zero occurences: ', transformer.nnz)

Amount of Non-Zero occurences:  16


In [17]:
print ('sparsity: %.2f%%' % (100.0 * transformer.nnz /
(transformer.shape[0] * transformer.shape[1])))

sparsity: 22.86%


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(transformer)

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.47140452],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ],
       [0.        , 0.47140452, 0.        , 0.        , 1.        ]])

#จะเห็นว่า Document 1 และ 4 มีความเหมือนกันอยู่ที่ 0.471 โดยที่หากค่าเป็น 1 แสดงว่ามีความเหมือนมากที่สุดและ 0 คือไม่มีความเหมือนกันเลย (มีคำว่า Universal Studio เหมือนกัน)

# Config tf-idf

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=False,).fit(transformer)

#show ค่า idf

In [21]:
tfidf_transformer.idf_

array([2.60943791, 2.60943791, 2.60943791, 2.60943791, 2.60943791,
       2.60943791, 2.60943791, 2.60943791, 2.60943791, 2.60943791,
       2.60943791, 1.91629073, 2.60943791, 1.91629073])

#จะเห็นว่าซ้ำกัน 2 คำคือ universal และ studio ที่เหลือไม่ซ้ำกันมีค่า 2.6
จะมี 2 คำคือคำว่า Universal และ Studio ที่มีค่าน้อยกว่า IDF แสดงให้เห็นว่าสองคำนี้มีความเป็น Unique น้อยกว่าเพราะมีการใช้ซ้ำใน Context อื่น

In [ ]:
#tf(t) = จำนวนที่ปรากฏใน document/จำนวนคำทั้งหมดใน document
#idf(d, t) = ln(จำนวน document ทั้งหมด/จำนวนที่ปรากฏใน Document) + 1
#idf(d, t) = ln [ (1 + จำนวน document) / (1 + จำนวนที่ปรากฏใน Document) ] + 1
#Parameter smoot=true => ln((1+5)/(1+1))+1 = 2.09861229
#Parameter smoot=false=>ln(5)+1 = 2.60943791
#บางนิยามใช้ log() บางนิยามใช้ ln()

In [22]:
from sklearn.naive_bayes import MultinomialNB

# เตรียม Class ในรูปแบบ array เหมือน arr_text

In [23]:
arr_class=data.iloc[:]['class'].values
arr_class

array(['london', 'japan', 'china', 'thailand', 'usa'], dtype=object)

# เตรียม train จาก transformer

In [24]:
messages_tfidf = tfidf_transformer.transform(transformer)

In [25]:
print (messages_tfidf.shape)

(5, 14)


In [26]:
messages_tfidf.toarray()

array([[0.        , 0.57735027, 0.        , 0.        , 0.57735027,
        0.        , 0.        , 0.57735027, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.44373957, 0.        , 0.44373957, 0.44373957,
        0.44373957, 0.32586866, 0.        , 0.32586866],
       [0.70710678, 0.        , 0.        , 0.70710678, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.70710678, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.69360936, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.50936532, 0.        , 0.50936532]])

# สร้าง Model

In [27]:
detect_model = MultinomialNB().fit(messages_tfidf,arr_class)

# การ Predict
สร้าง ตัว test
จะ test ด้วยคำว่า london bridge is falling down

In [28]:
arr_text[0]

'London bridge is falling down'

In [29]:
t0=vect.transform([arr_text[0]])

#แปลงเป็น vector

In [30]:
t0.toarray()

array([[0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [31]:
print(t0)

  (0, 1)	1
  (0, 4)	1
  (0, 7)	1


#นำ data ทั้งหมดที่ train ไป test มันก็ควรจะตรง(Expected คือ 100%)

In [47]:
all_predictions = detect_model.predict(messages_tfidf)

In [48]:
print(all_predictions)

['london' 'japan' 'china' 'thailand' 'usa']


# ดูค่า Performance

In [50]:
from sklearn.metrics import classification_report
print (classification_report(arr_class, all_predictions))

              precision    recall  f1-score   support

       china       1.00      1.00      1.00         1
       japan       1.00      1.00      1.00         1
      london       1.00      1.00      1.00         1
    thailand       1.00      1.00      1.00         1
         usa       1.00      1.00      1.00         1

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5



# ลอง test ด้วยคำใหม่ 

In [51]:
thaitest = 'Hello chiangmai I am siam'

In [52]:
vectmp = vect.transform([thaitest])

In [53]:
vectmp.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [54]:
tftmp = tfidf_transformer.transform(vectmp)

In [56]:
print(tftmp)

  (0, 2)	1.0


In [57]:
detect_model.predict(tftmp)

array(['thailand'], dtype='<U8')

# result

In [59]:
thaitest = 'Hello chiangmai I am siam'
vectmp = vect.transform([thaitest])
tftmp = tfidf_transformer.transform(vectmp)
detect_model.predict(tftmp)

array(['thailand'], dtype='<U8')

In [60]:
thaitest = 'Wellcome to beijing'
vectmp = vect.transform([thaitest])
tftmp = tfidf_transformer.transform(vectmp)
detect_model.predict(tftmp)


array(['china'], dtype='<U8')

In [ ]:
###Save model to file

#>>>> import pickle
#>>>> pickle.dump(detect_model,open("/tmp/nb.md",'wb'))

###Load model from file

#>>> import pickle
#>>> nb_model = pickle.load(open('/tmp/nb.md','rb'))
#>>> nb_model
#MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
